# PyTorch自动微分
autograd包是PyTorch中所有神经网络的核心。首先让我们简单地介绍它，然后我们将会去训练我们的第一个神经网络。**该autograd软件包为Tensors上的所有操作提供自动微分**。它是一个由运行定义的框架，这就意味着以代码运行方式定义你的向后传播，并且每次迭代都可以不同。
## Tensor
torch.Tensor是包的核心类。**如果将其属性.requires_grad设置为True**，则会开始跟踪对tensor的所有操作。完成计算后，你可以**使用.backward()来自动计算所有梯度**。该张量的梯度积累到.grad属性中。  

**要停止tensor历史记录的跟踪，你可以调用.detach()**，它将其与计算历史记录分离，并防止将来的计算被跟踪。  

要停止跟踪历史记录（和使用内存），你还可以将代码块使用 **with torch.no_grad()** 包装起来。在评估模型时，这是特别有用的，因为模型在训练阶段具有requires_grad = True 的可训练参数有利于调参，但是在评估阶段，我们并不需要梯度来进行调参。  

还有一个类对于autograd实现非常重要那就是Function。Tensor和Function相互连接并构建一个非循环图，它保存整个完整的计算过程的历史信息。**每个张量都有一个.grad_fn属性保存着创建了张量的Function的引用**，（如果是自己创建张量，则grad_fn=None）。  

如果你想**计算导数，可以调用Tensor.backward()**。如果Tensor是标量（即它包含一个元素数据），则不需要指定任何参数backward()，但如果它有更多元素，则需要指定gradient参数来指定张量的形状。

In [1]:
import torch

创建一个张量，设置requires_grad=True 来跟踪与它相关的计算。

In [2]:
x = torch.ones(2,2,requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


针对张量进行一个加法操作。

In [3]:
y = x + 3
print(y)

tensor([[4., 4.],
        [4., 4.]], grad_fn=<AddBackward0>)


根据结果可以看出来，y中有了grad_fn。  
下面我们输出来看看：

In [4]:
print(y.grad_fn)

针对y进行更多地操作：

In [5]:
z = y * y * 3
out = z.mean()
print(z)
print(out)

tensor([[48., 48.],
        [48., 48.]], grad_fn=<MulBackward0>)
tensor(48., grad_fn=<MeanBackward0>)


.requires_grad()会改变张量的requires_grad标记。输入的标记默认为False，如果没有提供相应的参数。

In [6]:
a = torch.randn(2,2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)  # 由于requires_grad为False，因此在使用Function计算得到的结果grad_fn为None

a.requires_grad_(True) # 设置requires_grad为True
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

False
None
True


## Gradient
我们现在向后传播，因为输出包含了一个标量，out.backward()等同于out.backward(torch.tensor(1.))。

In [7]:
out.backward()

打印梯度d(out)/dx

In [8]:
print(x.grad)

tensor([[6., 6.],
        [6., 6.]])


现在看一个雅克比向量积的例子：

In [9]:
x = torch.randn(3,requires_grad=True)
print(x)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([1.0359, 0.1405, 0.2498], requires_grad=True)
tensor([1060.7249,  143.8998,  255.8287], grad_fn=<MulBackward0>)


现在这种情况下，y不再是一个标量。torch.autograd不能直接计算整个雅克比，但是如果我们只想要雅克比向量积，只需要简单的传递向量给backward作为参数。

In [10]:
v = torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


你也可以将代码包裹在with torch.no_grad()，来停止对从跟踪历史中的.requires_grad=True的张量自动求导。

In [11]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False
